In [9]:
import logging
import os
import pickle
import time
import statistics
from abc import abstractmethod

import colorsys
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import hdbscan

from sklearn.cluster import OPTICS, cluster_optics_dbscan
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from sklearn.metrics import silhouette_score, silhouette_samples
import statistics

from sklearn.neighbors import LocalOutlierFactor

from sklearn.svm import OneClassSVM




In [10]:
f0 = 'no anomaly.csv'
f1 = 'engine.csv'
f2 = 'elevator.csv'
f3 = 'aileron.csv'
f4 = 'rudder.csv'

base_df = pd.read_csv(f0, header=0, index_col=None)
engine1_df = pd.read_csv(f1, header=0, index_col=None)
elevator1_df = pd.read_csv(f2, header=0, index_col=None)
aileron1_df = pd.read_csv(f3, header=0, index_col=None)
rudder1_df = pd.read_csv(f4, header=0, index_col=None)

df_list = [base_df, engine1_df, elevator1_df, aileron1_df, rudder1_df]

for df in df_list:

    df['airspeedchange'] = df['Airspeed']- df['Airspeed'].shift(1)
    df['airspeedchange'].fillna(0, inplace=True)

anomaly_list = elevator1_df['Anomaly'].tolist()

base_df = base_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]
engine = engine1_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]
clustering_elevator1_df = elevator1_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]
clustering_aileron_df = aileron1_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]
clustering_rudder_df = rudder1_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]

In [11]:
anomaly_list.count(1)

204

In [12]:
#train the 5 models

kmeans = KMeans(init="random", n_clusters=2, n_init=10, random_state=1)
kmeans.fit(base_df)

lof = LocalOutlierFactor(n_neighbors=14, contamination="auto",novelty=True)
lof.fit(base_df)

nu = 0.1
kernel = 'rbf'
gamma = 'scale'

model = OneClassSVM(nu=nu, kernel=kernel, gamma=gamma)
model.fit(base_df)

db = DBSCAN(eps = 0.78).fit(base_df)

optics_clustering = OPTICS(min_samples = 50).fit(base_df)



In [13]:
def rule_checker(datapoint):
    rules_broken = 0
    if datapoint["Roll"] > 47.3 or datapoint["Roll"] < -50:
        rules_broken += 1

    if datapoint["Pitch"] > 22.93 or datapoint["Pitch"] < -19.3:
        rules_broken += 1

    if datapoint["Yaw"] > 179.2 or datapoint["Yaw"] < -177.3:
        rules_broken += 1

    if datapoint["Airspeed"] > 24.29 or datapoint["Airspeed"] < 12.48:
        rules_broken += 1

    if datapoint["VelocityX"] > 19.73 or datapoint["VelocityX"] < -17.05:
        rules_broken += 1

    if datapoint["VelocityY"] > 21.4 or datapoint["VelocityY"] < -18.26:
        rules_broken += 1

    if datapoint["VelocityZ"] > 5.3 or datapoint["VelocityZ"] < -8.6:
        rules_broken += 1

    return rules_broken

<h3>ELEVATOR<h3>

In [14]:
elevator_rule_alerts = []

for i in range (len(elevator1_df)):
    alerts = 0

    alerts += (rule_checker(elevator1_df.iloc[i]))
    
    elevator_rule_alerts.append(alerts)
    
count = 0
for value in elevator_rule_alerts:
    if value >0:
        count +=1


#predict for elevator

elevator_kmeans = []
elevator_lof = []
elevator_svm = []
elevator_db = []
elevator_optics = []

base_df2 = base_df.copy()
base_df2['cluster'] = kmeans.labels_
elevator1_df2 = clustering_elevator1_df.copy()
elevator1_df2['cluster'] = kmeans.predict(clustering_elevator1_df)

centroids = kmeans.cluster_centers_

original_distances = []
for i in range(len(base_df)):
    centroid = centroids[int(base_df2.iloc[i]['cluster'])]
    distance = np.sqrt(np.sum((base_df.iloc[i] - centroid)**2))
    original_distances.append(distance)

original_mean = sum(original_distances)/len(original_distances)

original_std = np.std(original_distances)

threshold = original_mean + 3* original_std

for i in range(len(clustering_elevator1_df)):
    centroid = centroids[int(elevator1_df2.iloc[i]['cluster'])]
    distance = np.sqrt(np.sum((clustering_elevator1_df.iloc[i] - centroid)**2))
    if distance > threshold:
        elevator_kmeans.append(-1)
    else:
        elevator_kmeans.append(1)


for i in range(clustering_elevator1_df.shape[0]):
    prediction = lof.predict(clustering_elevator1_df.iloc[[i]])
    for value in prediction:
        elevator_lof.append(value)

for i in range(clustering_elevator1_df.shape[0]):
    prediction = model.predict(clustering_elevator1_df.iloc[[i]])
    for value in prediction:
        elevator_svm.append(value)

for value in db.fit_predict(clustering_elevator1_df):
    elevator_db.append(value)

for value in optics_clustering.fit_predict(clustering_elevator1_df):
    elevator_optics.append(value)


#get predictions for elevator using voting

final_elevator_predictions = []

for i in range (len(elevator1_df)):
    predictions = [elevator_kmeans[i], elevator_lof[i], elevator_svm[i], elevator_db[i], elevator_optics[i]]
    final_elevator_predictions.append(max(set(predictions), key=predictions.count))
    
overall_elevator_count = 0

for i in range(len(elevator1_df)):
    if final_elevator_predictions[i] <0 or elevator_rule_alerts[i] > 0:
        overall_elevator_count+=1

print("For elevator, out of the", len(elevator1_df), "anomalies", overall_elevator_count ,"were detected, which is", overall_elevator_count/len(elevator1_df)*100,"%")


/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
/Users/ivantan97/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warni

For elevator, out of the 2506 anomalies 2506 were detected, which is 100.0 %


In [15]:
unsupervised_count = 0

for i in range(len(elevator1_df)):
    if final_elevator_predictions[i] < 0:
        unsupervised_count += 1

print(unsupervised_count)


2194


In [16]:
rule_count = 0

for i in range(len(elevator1_df)):
    if elevator_rule_alerts[i] > 0:
        rule_count += 1

print(rule_count)

2506
